# 네이버 뉴스 가져오기

<br>

2020년1월1일 ~ 2020년 4월 14일 데이터 가져오기

'기사 더보기' 누르기 전 -> 기사가 몇개 없음

=> 동적으로 변경

<br>

동적으로 변경

In [1]:
import urllib.request
from bs4 import BeautifulSoup # 특정 웹 페이지의 HTML을 스크랩하여 데이터를 수집할 때 사용
import pandas as pd
import numpy as np
import re

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from bs4 import BeautifulSoup
# import time
from tqdm import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
# ! pip install lxml
# ! pip install html5lib

In [3]:
# 날짜 가져오기
# 시작 날짜와 끝 날짜 설정
# import datetime
# start_date = datetime.date(2020, 1, 1)
# end_date = datetime.date(2020, 4, 14)

In [4]:
# # 웹 드라이버 시작
# driver = webdriver.Chrome()
# df_2020 = pd.DataFrame()

# # 네이버 뉴스 페이지로 이동
# current_date = start_date
# while current_date <= end_date:
#     formatted_date = current_date.strftime("%Y%m%d")

#     driver.get('https://news.naver.com/breakingnews/section/100/265?date={}'.format(formatted_date))

#     # '기사 더보기'없을 때까지
#     try:
#         while True:
#             elem1 = driver.find_element(By.CLASS_NAME, "section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON")
#             elem1.click()
#             time.sleep(1)  # 대기
#     except Exception as e:
#         print(e)



#     # 페이지 소스 가져오기
#     page_source = driver.page_source

#     # BeautifulSoup을 사용하여 페이지 소스를 파싱
#     soup = BeautifulSoup(page_source, 'html.parser')

#     # 제목 가져오기
#     titles = soup.select('div.section_latest > div > div.section_latest_article._CONTENT_LIST._PERSIST_META > \
#                               div > ul > li > div > div > div.sa_text > a > strong.sa_text_strong')

#     # 링크 가져오기
#     links = soup.select('div.section_latest > div > div.section_latest_article._CONTENT_LIST._PERSIST_META > \
#                               div > ul > li > div > div > div.sa_text > a')

#     title_lst = []
#     content_lst = []

#     # 링크로 이동해서 내용 가져오기
#     for title, link in tqdm(zip(titles, links)):
#         article_link = link['href'] # 링크 가져오기
#         driver.get(article_link) # 링크로 이동

#         # 페이지 소스 가져오기
#         article_page_source = driver.page_source
#         article_soup = BeautifulSoup(article_page_source, 'lxml')

#         # 내용 가져오기
#         content = article_soup.select_one('div.ct_wrap > div.ct_scroll_wrapper > div#newsct > div#ct > div#contents > \
#                             div#newsct_article > article#dic_area')

#         if content:
#             content_lst.append(content.text.strip())
#         else:
#             content_lst.append('No content available') # 내용이 없는 경우 예외 처리

#         title_lst.append(title.string)


#     # 데이터 저장
#     df = pd.DataFrame({'title' : title_lst, 'content' : content_lst})
#     df_2020 = pd.concat([df_2020,df], ignore_index=True)

#     current_date += datetime.timedelta(days=1)

# # 웹 드라이버 종료
# driver.quit()

# df_2020.to_csv('2020_234_news.csv', index = False)

In [5]:
# df_2020_1 = pd.read_csv('./2020_1_news.csv')
# df_2020_234 = pd.read_csv('./2020_234_news.csv')

In [6]:
# df_2020_1
# df_2020_234

In [7]:
# df_news = pd.concat([df_2020_1, df_2020_234])
# df_news.to_csv('df_news.csv', index = False)

# 전처리

In [8]:
df_news = pd.read_csv('./data/news_2020.csv')

df_news

,title,content
0,"유시민 ""검찰도 사법도 썩었지"" vs 진중권 ""유시민 망상, 대중은 현실로 믿어""",JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁\n\n...
1,특정 정당 맹목적 지지 없어…진보진영 유리 장담 못해 [신년 기획 - 21대 총선 분석],"ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련 ..."
2,"한국당 ""울산시장 선거 정황 의심""…추미애 고발",[머니투데이 한고은 기자] \n\n\n\n(서울=뉴스1) 임세영 기자 = 곽상도 자...
3,"한국당, 추미애 檢 고발..""선거법 위반""","곽상도 친문농단특위 위원장, 1일 고발""지방선거 전 송철호에 靑행정관 소개""""공무상..."
4,"文의장 ""검찰·선거개혁 단초 열어 보람…소명 다할 것""","""공수처는 金·盧·文 세 대통령의 숙원이자 꿈"" 한국당 '아빠 찬스, 아들 공천'..."
...,...,...
98225,‘세월호 막말’ 닷새만에… 차명진 제명,"통합당, 긴급 최고위서 직권 의결… 수도권 전체 판세에 악영향 판단강원선 김진태 선..."
98226,정봉주 “나를 개쓰레기 취급” 與 막말 비난,논란 일자 “부적절한 표현” 사과\n\n\n\n열린민주당 정봉주 최고위원(사진)이 ...
98227,민주당 “아직 2% 부족” vs 통합당 “100석도 위태”,이해찬 “수도권 121석중 70곳 경합”박형준 “개헌 저지선도 무너질 위기” 김종인...
98228,"[총선2020] 범여권, '오세훈만 떨어뜨리면 된다' 계산 섰나","민주당 핵심 당직자들, 한날 광진을 지원방문'촛불시민모임' '대진연' 등도 계속해 ..."


In [9]:
df_news[df_news['content'] == 'No content available']

,title,content
4149,"[21대 총선] 광주 동남갑, '3선 장병완' 민주당 도전 막아낼까",No content available
4296,"'암벽여제' 김자인, 남편 민주당 입당에 ""믿고 응원""",No content available
5594,"'욱토크' 박지원 ""문재인 정부, 제일 못하는 건 청년실업 문제""",No content available
5596,"'욱토크' 박지원 ""홍준표, 친분 있지만 사이 나쁘다…나한테 못 당해""",No content available
5677,"정민당 ""'음원사재기' 송하예 소속사 고발""...송하예 측 ""사실무근·법적대응""",No content available
...,...,...
88799,'황교안을 향한 손짓',No content available
88976,이낙연 위원장 '황기철 후보 믿어주세요',No content available
92764,유세하는 이낙연,No content available
96636,"연매협 ""김근태 후보, 연예인 이용 이슈몰이…직접 소명해야""",No content available


## 필요없는 문자 제거

In [10]:
df_news = df_news[df_news['content'] != 'No content available']
df_news = df_news.drop_duplicates(subset = ['title'])
df_news = df_news.copy()
df_news

,title,content
0,"유시민 ""검찰도 사법도 썩었지"" vs 진중권 ""유시민 망상, 대중은 현실로 믿어""",JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁\n\n...
1,특정 정당 맹목적 지지 없어…진보진영 유리 장담 못해 [신년 기획 - 21대 총선 분석],"ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련 ..."
2,"한국당 ""울산시장 선거 정황 의심""…추미애 고발",[머니투데이 한고은 기자] \n\n\n\n(서울=뉴스1) 임세영 기자 = 곽상도 자...
3,"한국당, 추미애 檢 고발..""선거법 위반""","곽상도 친문농단특위 위원장, 1일 고발""지방선거 전 송철호에 靑행정관 소개""""공무상..."
4,"文의장 ""검찰·선거개혁 단초 열어 보람…소명 다할 것""","""공수처는 金·盧·文 세 대통령의 숙원이자 꿈"" 한국당 '아빠 찬스, 아들 공천'..."
...,...,...
98225,‘세월호 막말’ 닷새만에… 차명진 제명,"통합당, 긴급 최고위서 직권 의결… 수도권 전체 판세에 악영향 판단강원선 김진태 선..."
98226,정봉주 “나를 개쓰레기 취급” 與 막말 비난,논란 일자 “부적절한 표현” 사과\n\n\n\n열린민주당 정봉주 최고위원(사진)이 ...
98227,민주당 “아직 2% 부족” vs 통합당 “100석도 위태”,이해찬 “수도권 121석중 70곳 경합”박형준 “개헌 저지선도 무너질 위기” 김종인...
98228,"[총선2020] 범여권, '오세훈만 떨어뜨리면 된다' 계산 섰나","민주당 핵심 당직자들, 한날 광진을 지원방문'촛불시민모임' '대진연' 등도 계속해 ..."


In [11]:
df_news['content'] = df_news['content'].str.replace('\n', ' ')
df_news['content'] = df_news['content'].str.replace('/', ' ')


df_news

,title,content
0,"유시민 ""검찰도 사법도 썩었지"" vs 진중권 ""유시민 망상, 대중은 현실로 믿어""",JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁 ...
1,특정 정당 맹목적 지지 없어…진보진영 유리 장담 못해 [신년 기획 - 21대 총선 분석],"ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련 ..."
2,"한국당 ""울산시장 선거 정황 의심""…추미애 고발",[머니투데이 한고은 기자] (서울=뉴스1) 임세영 기자 = 곽상도 자유한국당...
3,"한국당, 추미애 檢 고발..""선거법 위반""","곽상도 친문농단특위 위원장, 1일 고발""지방선거 전 송철호에 靑행정관 소개""""공무상..."
4,"文의장 ""검찰·선거개혁 단초 열어 보람…소명 다할 것""","""공수처는 金·盧·文 세 대통령의 숙원이자 꿈"" 한국당 '아빠 찬스, 아들 공천'..."
...,...,...
98225,‘세월호 막말’ 닷새만에… 차명진 제명,"통합당, 긴급 최고위서 직권 의결… 수도권 전체 판세에 악영향 판단강원선 김진태 선..."
98226,정봉주 “나를 개쓰레기 취급” 與 막말 비난,논란 일자 “부적절한 표현” 사과 열린민주당 정봉주 최고위원(사진)이 12일 ...
98227,민주당 “아직 2% 부족” vs 통합당 “100석도 위태”,이해찬 “수도권 121석중 70곳 경합”박형준 “개헌 저지선도 무너질 위기” 김종인...
98228,"[총선2020] 범여권, '오세훈만 떨어뜨리면 된다' 계산 섰나","민주당 핵심 당직자들, 한날 광진을 지원방문'촛불시민모임' '대진연' 등도 계속해 ..."


## 후보 이름을 당으로 변환

In [12]:
name_df = pd.read_csv('./data/제21대_국회의원선거_후보자_명단.csv')

name_df

,후보,당선여부,소속 정당
0,정혜선,0,민생당
1,이내훈,0,민생당
2,김정화,0,민생당
3,김종구,0,민생당
4,장정숙,0,민생당
...,...,...,...
307,김호일,0,한국복지당
308,정월자,0,한국복지당
309,이정복,0,한국복지당
310,장소영,0,홍익당


In [13]:
name_party_dic = dict(zip(name_df['후보'], name_df['소속 정당']))

name_party_dic

{'정혜선': '민생당',
 '이내훈': '민생당',
 '김정화': '민생당',
 '김종구': '민생당',
 '장정숙': '민생당',
 '이관승': '민생당',
 '최도자': '민생당',
 '황한웅': '민생당',
 '서진희': '민생당',
 '한지호': '민생당',
 '고연호': '민생당',
 '김지환': '민생당',
 '문정선': '민생당',
 '손학규': '민생당',
 '한성숙': '민생당',
 '이봉원': '민생당',
 '이시은': '민생당',
 '김선미': '민생당',
 '이숙자': '민생당',
 '조환기': '민생당',
 '추민아': '민생당',
 '윤주경': '미래한국당',
 '윤창현': '미래한국당',
 '한무경': '미래한국당',
 '이종성': '미래한국당',
 '조수진': '미래한국당',
 '조태용': '미래한국당',
 '정경희': '미래한국당',
 '신원식': '미래한국당',
 '조명희': '미래한국당',
 '박대수': '미래한국당',
 '김예지': '미래한국당',
 '지성호': '미래한국당',
 '이영': '미래한국당',
 '최승재': '미래한국당',
 '전주혜': '미래한국당',
 '정운천': '미래한국당',
 '서정숙': '미래한국당',
 '이용': '미래한국당',
 '허은아': '미래한국당',
 '노용호': '미래한국당',
 '최영희': '미래한국당',
 '우신구': '미래한국당',
 '김은희': '미래한국당',
 '하재주': '미래한국당',
 '박현정': '미래한국당',
 '권신일': '미래한국당',
 '백현주': '미래한국당',
 '남영호': '미래한국당',
 '문혜정': '미래한국당',
 '박대성': '미래한국당',
 '정선미': '미래한국당',
 '김보람': '미래한국당',
 '권순영': '미래한국당',
 '이종헌': '미래한국당',
 '서안순': '미래한국당',
 '김영근': '미래한국당',
 '김경애': '미래한국당',
 '신민아': '미래한국당',
 '김란숙': '미래한국당',
 '신현영': '더불어시민당',
 '김경만'

In [14]:
# title 열의 각 값에 대해 사람 이름을 당의 이름으로 변경

for idx, row in tqdm(df_news.iterrows()):
    for name, party in name_party_dic.items():
        if name in row['title']:
            df_news.at[idx, 'title'] = df_news.at[idx, 'title'].replace(name, party)
        elif name in row['content']:
            df_news.at[idx, 'content'] = df_news.at[idx, 'content'].replace(name, party)

79410it [00:46, 1690.11it/s]


In [15]:
df_news

,title,content
0,"유시민 ""검찰도 사법도 썩었지"" vs 진중권 ""유시민 망상, 대중은 현실로 믿어""",JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁 ...
1,특정 정당 맹목적 지지 없어…진보진영 유리 장담 못해 [신년 기획 - 21대 총선 분석],"ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련 ..."
2,"한국당 ""울산시장 선거 정황 의심""…추미애 고발",[머니투데이 한고은 기자] (서울=뉴스1) 임세영 기자 = 곽상도 자유한국당...
3,"한국당, 추미애 檢 고발..""선거법 위반""","곽상도 친문농단특위 위원장, 1일 고발""지방선거 전 송철호에 靑행정관 소개""""공무상..."
4,"文의장 ""검찰·선거개혁 단초 열어 보람…소명 다할 것""","""공수처는 金·盧·文 세 대통령의 숙원이자 꿈"" 한국당 '아빠 찬스, 아들 공천'..."
...,...,...
98225,‘세월호 막말’ 닷새만에… 차명진 제명,"통합당, 긴급 최고위서 직권 의결… 수도권 전체 판세에 악영향 판단강원선 김진태 선..."
98226,정봉주 “나를 개쓰레기 취급” 與 막말 비난,논란 일자 “부적절한 표현” 사과 열린민주당 정봉주 최고위원(사진)이 12일 ...
98227,민주당 “아직 2% 부족” vs 통합당 “100석도 위태”,이해찬 “수도권 121석중 70곳 경합”박형준 “개헌 저지선도 무너질 위기” 김종인...
98228,"[총선2020] 범여권, '오세훈만 떨어뜨리면 된다' 계산 섰나","민주당 핵심 당직자들, 한날 광진을 지원방문'촛불시민모임' '대진연' 등도 계속해 ..."


## 문장으로 나누기

In [16]:
df_news['content'] = df_news['content'].str.split('.')

df_news

,title,content
0,"유시민 ""검찰도 사법도 썩었지"" vs 진중권 ""유시민 망상, 대중은 현실로 믿어""",[JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁 ...
1,특정 정당 맹목적 지지 없어…진보진영 유리 장담 못해 [신년 기획 - 21대 총선 분석],"[ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련..."
2,"한국당 ""울산시장 선거 정황 의심""…추미애 고발",[[머니투데이 한고은 기자] (서울=뉴스1) 임세영 기자 = 곽상도 자유한국...
3,"한국당, 추미애 檢 고발..""선거법 위반""","[곽상도 친문농단특위 위원장, 1일 고발""지방선거 전 송철호에 靑행정관 소개""""공무..."
4,"文의장 ""검찰·선거개혁 단초 열어 보람…소명 다할 것""","[""공수처는 金·盧·文 세 대통령의 숙원이자 꿈"" 한국당 '아빠 찬스, 아들 공천..."
...,...,...
98225,‘세월호 막말’ 닷새만에… 차명진 제명,"[통합당, 긴급 최고위서 직권 의결… 수도권 전체 판세에 악영향 판단강원선 김진태 ..."
98226,정봉주 “나를 개쓰레기 취급” 與 막말 비난,[논란 일자 “부적절한 표현” 사과 열린민주당 정봉주 최고위원(사진)이 12일...
98227,민주당 “아직 2% 부족” vs 통합당 “100석도 위태”,[이해찬 “수도권 121석중 70곳 경합”박형준 “개헌 저지선도 무너질 위기” 김종...
98228,"[총선2020] 범여권, '오세훈만 떨어뜨리면 된다' 계산 섰나","[민주당 핵심 당직자들, 한날 광진을 지원방문'촛불시민모임' '대진연' 등도 계속해..."


In [ ]:
# # '@'가 포함된 값 이전까지 가져오기
# from tqdm import tqdm

# # 새로운 열을 생성하기 위해 빈 리스트 초기화
# new_content = []

# for i in tqdm(range(len(df_news))):
#     res = []
#     for j in range(len(df_news.iloc[i, 1])):
#         if not (re.search("[a-zA-Z]", df_news.iloc[i, 1][j]) or '@' in df_news.iloc[i, 1][j] or '<' in df_news.iloc[i, 1][j] or '>' in df_news.iloc[i, 1][j]):
#             res.append(df_news.iloc[i, 1][j])
#         else:
#             continue
#     # 각 행에 대한 결과를 빈 리스트에 추가
#     new_content.append(res)

# # 새로운 열로 할당
# df_news['new_content'] = new_content

'@'가 포함된 값 이전까지 가져오기

In [ ]:
# # 이메일이 먼저 나오는 경우 일단 넘어가서 가져오기

# from tqdm import tqdm

# # 새로운 열을 생성하기 위해 빈 리스트 초기화
# new_content = []

# for i in tqdm(range(len(df_news))):
#     res = []
#     for j in range(len(df_news.iloc[i, 1])):
#         if '@' in df_news.iloc[i, 1][j]:
#             if j <= (len(df_news.iloc[i, 1]) // 2):
#                 # '@'가 전체 리스트의 절반보다 앞에 있는 경우
#                 res.extend(df_news.iloc[i, 1][j:])
#             else:
#                 # '@'가 전체 리스트의 절반보다 뒤에 있는 경우
#                 res.extend(df_news.iloc[i, 1][:j])
#                 break  # 뒤에 있는 부분만 가져온 후 반복문 종료

#     # 각 행에 대한 결과를 빈 리스트에 추가
#     new_content.append(res)

# # 새로운 열로 할당
# df_news['new_content'] = new_content

In [46]:
# from tqdm import tqdm

# # 새로운 열을 생성하기 위해 빈 리스트 초기화
# new_content = []

# for i in tqdm(range(len(df_news))):
#     res = []
#     for j in range(len(df_news.iloc[i, 1])):
#         if '@' in df_news.iloc[i, 1][j]:
#             continue  # '@'가 있는 리스트는 건너뜁니다.
#         res.append(df_news.iloc[i, 1][j])

#     # 각 행에 대한 결과를 빈 리스트에 추가
#     new_content.append(res)

# # 새로운 열로 할당
# df_news['new_content'] = new_content


100%|███████████████████████████████████| 79410/79410 [00:14<00:00, 5617.86it/s]


In [ ]:
# df_news['new_content'][10]

['공무상 비밀누설죄 및 공직선거법 위반 혐의"文 선거개입 수사 방해하려 임명 서두르는 듯""철저한 검찰수사로 선거개입 진실 밝혀져야"    [서울=뉴시스]김명원 기자 = 추미애 법무부장관 후보자가 30일 오후 서울 여의도 국회 법제사법위원회에서 열린 법무부장관 후보자 인사청문회에 참석해 의원들의 질의를 경청하며 가져온 음료를 마시고 있다',
 '  2019',
 '12',
 '30',
 'com [서울=뉴시스] 이승주 기자 = 자유한국당이 1일 선거개입 의혹을 제기하며 추미애 법무부장관 후보자를 검찰에 고발했다',
 '친문농단진상조사특위 총괄본부장을 맡고 있는 곽상도 한국당 의원은 이날 보도자료를 통해 "청와대에 이어 추미애 전 민주당 대표의 선거개입 정황이 의심된다"며 "추 전 대표를 공무상 비밀누설죄 및 공직선거법 위반 혐의로 고발했다"고 밝혔다',
 '곽 의원은 "지난 6·13선거와 관련 추 전 대표 측 관계자는 청와대 균형발전비서관실 소속 장환석 선임행정관을 송철호·송병기 등에게 소개했다고 한다"며 "이를 통해 송철호 측은 외부에 공개되지 않은 울산 관련 정보를 선거 공약으로 활용했다는 것이 언론에 밝혀졌다"고 했다',
 '그러면서 "문재인 대통령이 추 전 대표를 선거사범 등을 관장하는 주무부처인 법무부 장관으로 서둘러 임명하려는 것도, 장관도 아닌 후보자 신분으로 추 전 대표가 검사 인사 관련 보고를 받았다는 것도 청와대 선거 개입 관련 수사를 방해하려는 작업으로 볼 수밖에 없다"고 주장했다',
 '이어 "청와대의 법무부 장관 임명이 범죄 은폐 수단이 되어선 안 된다"며 "철저한 검찰 수사로 청와대의 선거 개입 의혹 진실이 낱낱이 밝혀져야 한다"고 강조했다',
 'com▶ K-Artprice 모바일 오픈! 미술작품 가격을 공개합니다▶ 뉴시스 채널 구독하고 에어팟 프로 받아가세요▶ 뉴시스 빅데이터 MSI 주가시세표 바로가기 <저작권자ⓒ 공감언론 뉴시스통신사',
 ' 무단전재-재배포 금지',
 '>']

# 긍정/부정 구분

In [19]:
party_names = [
    '더불어민주당', '미래통합당', '민생당', '정의당', '우리공화당', '민중당', '한국경제당', '국민의당', '친박신당', '열린민주당',
    '미래한국당', '더불어시민당', '가자코리아', '가자!평화인권당', '가자환경당', '공화당', '국가혁명배당금당', '국민새정당',
    '국민참여신당', '기독자유통일당', '기본소득당', '깨어있는시민연대당', '남북통일당', '노동당', '녹색당', '대한당', '대한민국당',
    '미래당', '미래민주당', '민중민주당', '새누리당', '시대전환', '여성의당', '우리당(2020년)', '자유당', '자유의새벽당',
    '중소자영업당', '직능자영업당', '충청의미래당', '통일민주당', '한국복지당', '한나라당', '홍익당', '자유한국당', '무소속'
]


당에 관한 기사만

In [20]:
df_news.head()

,title,content
0,"유시민 ""검찰도 사법도 썩었지"" vs 진중권 ""유시민 망상, 대중은 현실로 믿어""",[JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁 ...
1,특정 정당 맹목적 지지 없어…진보진영 유리 장담 못해 [신년 기획 - 21대 총선 분석],"[ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련..."
2,"한국당 ""울산시장 선거 정황 의심""…추미애 고발",[[머니투데이 한고은 기자] (서울=뉴스1) 임세영 기자 = 곽상도 자유한국...
3,"한국당, 추미애 檢 고발..""선거법 위반""","[곽상도 친문농단특위 위원장, 1일 고발""지방선거 전 송철호에 靑행정관 소개""""공무..."
4,"文의장 ""검찰·선거개혁 단초 열어 보람…소명 다할 것""","[""공수처는 金·盧·文 세 대통령의 숙원이자 꿈"" 한국당 '아빠 찬스, 아들 공천..."


In [29]:
# 당에 관한 기사만

party_sentences = []
party_lst = []

for data in tqdm(df_news['content'][:5]):
    res = []
    party_name = []
    print(data[0])
    
    for j in range(len(data)):
        for party in party_names:
            if party in data[j]:
                party_name.append(party)
                res.append(data[j])
                break  # 당 이름이 발견되면 해당 문장을 추가하고 내부 루프 종료
    party_sentences.append(res)
    party_lst.append(party_name)

df_news['party_content'] = party_sentences
df_news['party_name'] = party_lst

100%|██████████████████████████████████████████| 5/5 [00:00<00:00, 32463.65it/s]

JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁    유시민 사람사는세상 노무현재단 이사장과 진중권 전 동양대교수가 1일 오후 경기 고양시 일산 JTBC 스튜디오에서 열린 JTBC 신년특집 토론회에서 '한국 언론, 어디에 서있나'를 주제로 토론하고 있다
ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련 분주선거법 개정안은 만 18세 청소년도 투표나 선거운동이 가능하도록 규정했다
[머니투데이 한고은 기자]     (서울=뉴스1) 임세영 기자 = 곽상도 자유한국당 文정권 국정농단 3대 게이트 진상조사 총괄본부장이 17일 오후 서울 여의도 국회 정론관에서 기자회견을 갖고 김기현 전 울산시장 측근 비위 의혹 첩보 전달 과정을 설명하고 있다
곽상도 친문농단특위 위원장, 1일 고발"지방선거 전 송철호에 靑행정관 소개""공무상 비밀 누설죄·선거법 위반"    추미애 법무부 장관 후보자
"공수처는 金·盧·文 세 대통령의 숙원이자 꿈"  한국당 '아빠 찬스, 아들 공천' 공격에 "떳떳하고 당당, 자랑스럽다"    문희상<사진> 국회의장은 1일 공직선거법 개정안과 고위공직자범죄수사처(공수처) 법안이 국회 본회의를 통과한 것에 대해 "검찰개혁, 선거개혁이라는 두 가지 중요한 측면에서 촛불혁명 정신이 발현되는 단초를 열었고, 보람으로 생각한다"고 했다


In [49]:
df_news

,title,content,new_content,party_content,party_name
0,"유시민 ""검찰도 사법도 썩었지"" vs 진중권 ""유시민 망상, 대중은 현실로 믿어""",[JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁 ...,[JTBC 신년토론서 격돌조국 전 장관 사태와 '유시민의 알릴레오' 두고 논쟁 ...,[],[]
1,특정 정당 맹목적 지지 없어…진보진영 유리 장담 못해 [신년 기획 - 21대 총선 분석],"[ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련...","[ㆍ사상 첫 18세 투표 영향은신규 유입 유권자 약 53만명여야, 청소년 전략 마련...","[ 더불어민주당은 정책, 영입을 병행 추진하고 있다, 자유한국당은 선거연령 하향...","[더불어민주당, 자유한국당]"
2,"한국당 ""울산시장 선거 정황 의심""…추미애 고발",[[머니투데이 한고은 기자] (서울=뉴스1) 임세영 기자 = 곽상도 자유한국...,[[머니투데이 한고은 기자] (서울=뉴스1) 임세영 기자 = 곽상도 자유한국...,[[머니투데이 한고은 기자] (서울=뉴스1) 임세영 기자 = 곽상도 자유한국...,"[자유한국당, 자유한국당]"
3,"한국당, 추미애 檢 고발..""선거법 위반""","[곽상도 친문농단특위 위원장, 1일 고발""지방선거 전 송철호에 靑행정관 소개""""공무...","[곽상도 친문농단특위 위원장, 1일 고발""지방선거 전 송철호에 靑행정관 소개""""공무...",[ (사진=노진환 기자)[이데일리 김겨레 기자]자유한국당이 1일 추미애 법무부 장관...,[더불어민주당]
4,"文의장 ""검찰·선거개혁 단초 열어 보람…소명 다할 것""","[""공수처는 金·盧·文 세 대통령의 숙원이자 꿈"" 한국당 '아빠 찬스, 아들 공천...","[""공수처는 金·盧·文 세 대통령의 숙원이자 꿈"" 한국당 '아빠 찬스, 아들 공천...",[ 그러면서 자유한국당이 자신을 향해 아들 공천 문제 때문에 여당 편을 든다고 반발...,[자유한국당]
...,...,...,...,...,...
98225,‘세월호 막말’ 닷새만에… 차명진 제명,"[통합당, 긴급 최고위서 직권 의결… 수도권 전체 판세에 악영향 판단강원선 김진태 ...","[통합당, 긴급 최고위서 직권 의결… 수도권 전체 판세에 악영향 판단강원선 김진태 ...","[통합당, 긴급 최고위서 직권 의결… 수도권 전체 판세에 악영향 판단강원선 김진태 ...",[미래통합당]
98226,정봉주 “나를 개쓰레기 취급” 與 막말 비난,[논란 일자 “부적절한 표현” 사과 열린민주당 정봉주 최고위원(사진)이 12일...,[논란 일자 “부적절한 표현” 사과 열린민주당 정봉주 최고위원(사진)이 12일...,[논란 일자 “부적절한 표현” 사과 열린민주당 정봉주 최고위원(사진)이 12일...,"[더불어민주당, 열린민주당, 열린민주당, 열린민주당, 더불어시민당, 열린민주당]"
98227,민주당 “아직 2% 부족” vs 통합당 “100석도 위태”,[이해찬 “수도권 121석중 70곳 경합”박형준 “개헌 저지선도 무너질 위기” 김종...,[이해찬 “수도권 121석중 70곳 경합”박형준 “개헌 저지선도 무너질 위기” 김종...,[com 4·15총선을 이틀 남긴 13일 정치권은 미래통합당이 제기한 ‘여당...,"[미래통합당, 더불어민주당, 더불어시민당]"
98228,"[총선2020] 범여권, '오세훈만 떨어뜨리면 된다' 계산 섰나","[민주당 핵심 당직자들, 한날 광진을 지원방문'촛불시민모임' '대진연' 등도 계속해...","[민주당 핵심 당직자들, 한날 광진을 지원방문'촛불시민모임' '대진연' 등도 계속해...","[ 더불어민주당 이낙연 상임선거대책위원장, 이인영 원내대표, 박광온 최고위원 등 핵...","[더불어민주당, 미래통합당, 미래통합당]"


In [50]:
# 빈 리스트인 행의 개수를 세는 변수 초기화
empty_list_count = 0

# 데이터프레임 각 행에 대해 반복하여 빈 리스트인지 확인
for row in df_news['party_content']:
    if len(row) == 0:
        empty_list_count += 1

print("빈 리스트인 행의 개수:", empty_list_count)

빈 리스트인 행의 개수: 11768


=> 당에 속한 사람도 있을 것이니 이후 추가해서 다시 확인(사람 이름으로) 완료!!!!!

In [ ]:
! pip install transformers
! pip install torch

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# KoBERT 모델과 토크나이저 불러오기
model_name = "snunlp/KR-BERT-char16424"
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case = False)
model = BertForSequenceClassification.from_pretrained(model_name)

In [12]:
# from transformers import LongformerTokenizer, LongformerForSequenceClassification
# import torch
# from tqdm import tqdm

# # Longformer 모델과 토크나이저 불러오기
# model_name = 'allenai/longformer-base-4096'
# tokenizer = LongformerTokenizer.from_pretrained(model_name)
# model = LongformerForSequenceClassification.from_pretrained(model_name)

C:\Users\mit005\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


C:\Users\mit005\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mit005\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# # 입력 문장
# text = "너가 싫어"

# # 토큰화 및 패딩
# inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# # 모델로 예측
# outputs = model(**inputs)

# # 로짓 출력
# logits = outputs.logits

# # 예측 결과
# predictions = torch.argmax(logits, dim = 1)
# sentiment = "긍정" if predictions == 1 else "부정"

# print(f"입력 문장 : {text}")
# print(f"감정 분석 결과 : {sentiment}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


입력 문장 : 너가 싫어
감정 분석 결과 : 긍정


In [ ]:
# # 데이터프레임의 열에 있는 리스트의 각 요소를 처리하고 감정 분석 수행
# # 테스트
# a = df_news.iloc[0:10]

# for idx, row in a.iterrows():
#     content_list = row['party_content']
#     for content in content_list:
#         # 토큰화 및 패딩
#         inputs = tokenizer(content, return_tensors = "pt", padding = True, truncation = True)

#         # 모델로 예측
#         outputs = model(**inputs)

#         # 로짓 출력
#         logits = outputs.logits

#         # 예측 결과
#         predictions = torch.argmax(logits, dim  = 1)
#         sentiment = "긍정" if predictions == 1 else "부정"

#         print(f"입력 문장 : {content}")
#         print(f"감정 분석 결과 : {sentiment}")

입력 문장 :  더불어민주당은 정책, 영입을 병행 추진하고 있다
감정 분석 결과 : 긍정
입력 문장 :   자유한국당은 선거연령 하향을 반대했지만 법안 통과 후 청소년 전략 마련을 촉구하는 목소리가 높아지고 있다
감정 분석 결과 : 부정
입력 문장 : [머니투데이 한고은 기자]     (서울=뉴스1) 임세영 기자 = 곽상도 자유한국당 文정권 국정농단 3대 게이트 진상조사 총괄본부장이 17일 오후 서울 여의도 국회 정론관에서 기자회견을 갖고 김기현 전 울산시장 측근 비위 의혹 첩보 전달 과정을 설명하고 있다
감정 분석 결과 : 긍정
입력 문장 : 17 뉴스1   <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>자유한국당이 1일 추미애 법무부장관 후보자에 대한 선거개입 의혹을 제기하고, 추 후보자를 검찰에 고발했다
감정 분석 결과 : 부정
입력 문장 :  (사진=노진환 기자)[이데일리 김겨레 기자]자유한국당이 1일 추미애 법무부 장관 후보자가 더불어민주당 대표 시절 선거법을 위반했다며 검찰에 고발했다
감정 분석 결과 : 부정
입력 문장 :  그러면서 자유한국당이 자신을 향해 아들 공천 문제 때문에 여당 편을 든다고 반발한 것에 대해 "역사에서 어떻게 평가를 받아도 좋다
감정 분석 결과 : 긍정
입력 문장 :  출마지역으로는 문 대통령 사저가 있고 서형수 현 더불어민주당 의원이 불출마를 선언한 경남 양산을, 박영선 중소벤처기업부 장관 지역구인 서울 구로구을이 거론됐던 가운데 최종적으로는 구로구을 출마로 가닥을 잡은 기류다
감정 분석 결과 : 부정
입력 문장 : 윤 실장과 함께 청와대 내 총선 출마 예상자 명단에 이름을 올려온 고민정 대변인은 더불어민주당의 출마 요청을 받긴 했으나 불출마로 입장을 정리한 것으로 전해진다
감정 분석 결과 : 부정
입력 문장 :  사진=뉴스1자유한국당의 극렬한 반발 속 고위공직자범죄수사처(공수처)법이 국회 본회의를 통과한 것과 관련, 문희상 국회의장이 "내 소명이고 운명이었다"고 소회를 밝혔다
감정 분석 결과 : 부정


In [ ]:
df_news = df_news.reset_index(drop = True)

df_news

In [ ]:
# 전체
# 데이터프레임의 열에 있는 리스트의 각 요소를 처리하고 감정 분석 수행
res = []
for idx, row in tqdm(df_news.iterrows()):
    label_res = []
    content_list = row['party_content']
    for content in content_list:
        # 토큰화 및 패딩
        # 최대 입력 길이 설정
        max_length = 512

        # 토크나이저를 사용하여 입력 토큰화 및 패딩
        inputs = tokenizer(content, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

        # 모델로 예측
        outputs = model(**inputs)

        # 로짓 출력
        logits = outputs.logits

        # 예측 결과
        predictions = torch.argmax(logits, dim =1)
        sentiment = "긍정" if predictions == 1 else "부정"

        label_res.append(sentiment)
    res.append(label_res)

df_news['label'] = res

622it [24:07,  4.59s/it]

In [ ]:
df_news.to_csv('긍정부정.csv', index = False)